In [ ]:
%load_ext autoreload
%autoreload 2

import os
from src.video_preprocessing.download_videos.youtube_download import preprocess_video
from src.video_preprocessing.scene_detection.scene_detect import detect_scenes
from src.video_preprocessing.download_videos.download_utils import (
    transcribe_audio_files,
    extract_and_store_audio,
)
from src.ocr.pytesseract_image_to_text import extract_text_from_image
from sentence_transformers import SentenceTransformer
from src.text_embedder.embedder import text_to_embedding_transformer

from src.llm.ollama_implementation.ollama_experiment import (
    prompt_llm_summary,
    generate_caption_using_llava,
    prompt_llm_extensive_summary,
)
from src.video_preprocessing.download_videos.download_utils import (
    transcription_to_text,
    create_metadata,
)

from PIL import Image

import torch
from loguru import logger
import pickle

from src.clip.clip_model import CLIPEmbeddingsModel

import tqdm
from pathlib import Path
import os

In [ ]:
clip_model = CLIPEmbeddingsModel()


In [ ]:
# print pickle file 
with open("bio_3_3_th5.pickle", "rb") as file:
    data = pickle.load(file)

logger.info(f"new_pickle: {data}")

In [ ]:
# set image paths for the experiments
extracted_data_path = [data[key]['img_path'] for key in data.keys() if
                       'img_path' in data[key]]

clip_model.img_paths = None

clip_model.img_paths = extracted_data_path

In [2]:
for i in tqdm(range(len(clip_model.img_paths))):
    print(clip_model.img_paths[i])

NameError: name 'tqdm' is not defined

In [1]:
# Embedded with standard Tokenizer - short_llm_summary: OCR * Transcriptions * LLAVA 
# TODO: Need to get embedding with standard tokenizer for short_llm_summary


logger.info(f"Embedded with standard Tokenizer - clip_llm_summary: OCR * LLAVA * Transcriptions")

clip_model.text_embeddings = None

# Assuming that the pickle file has standard tokenizer embeddings 
extracted_data_text = [data[key]['ocr_extracted_text'] for key in data.keys() if
                       'ocr_extracted_text' in data[key]]

extracted_data_transcriptions = [data[key]['transcription'] for key in data.keys() if
                                 'transcription' in data[key]]

extracted_data_llava_result = [data[key]['llava_result'] for key in data.keys() if
                               'llava_result' in data[key]]

# iterate through the ocr text and llava results lists
from tqdm import tqdm

keyframes = {}

# iterate through the ocr text and llava results lists with a progress bar
for i in tqdm(range(len(clip_model.img_paths))):
    # only generate emb for the first 3 elements
    if i <3:
        ocr_text = extracted_data_text[i]
        transcription = extracted_data_transcriptions[i]
        llava_results = extracted_data_llava_result[i]
        
        short_llm_summary = prompt_llm_summary(
            slide_content=ocr_text,
            transcription=transcription,
            llava_output=llava_results,
        )
    
        extensive_summary = prompt_llm_extensive_summary(
            slide_content=ocr_text,
            transcription=transcription,
            llava_output=llava_results,
        )
        
        # these are the embeddings with a standard  tokenizer
        standard_text_embedding = clip_model.generate_dataset_embeddings_standard_tokenizer(
                short_llm_summary
        )
        extensive_text_embedding = clip_model.generate_dataset_embeddings_standard_tokenizer(
                extensive_summary
        )
        keyframe_num = "dummy"
        filepath = "dummy"
        timestamps = "dummy"
        clip_text_embedding = standard_text_embedding
        clip_image_embedding = "dummy"
        keyframe, keyframe_metadata = create_metadata(
            keyframe_num,
            filepath,
            timestamps,
            transcription,
            ocr_text,
            llava_results,
            short_llm_summary,
            extensive_summary,
            standard_text_embedding,
            extensive_text_embedding,
        )
        keyframes[keyframe] = keyframe_metadata
        # print(keyframes)

    # Save keyframes dictionary as Pickle

# Save with pickle
with open("data_standard_tokenizer.pickle", "wb") as file:
    pickle.dump(keyframes, file)


NameError: name 'logger' is not defined